In [ ]:
#importer les packages
# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"


import numpy as np
import pandas as pd

# To plot pretty figures
import matplotlib as mpl
import matplotlib.pyplot as plt


import keras
from keras.models import Sequential
from keras.layers import Conv2D, Dropout, Dense, Flatten, BatchNormalization, MaxPooling2D
from keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train_data = np.genfromtxt('/content/drive/My Drive/tp Appren/train_data.csv',delimiter=',')

In [ ]:
test_data = np.genfromtxt('/content/drive/My Drive/tp Appren/test_data.csv',delimiter=',')

In [ ]:
train_data.shape

In [ ]:
test_data.shape

In [ ]:
X_train_full = train_data[:,:-1]
y_train_full = train_data[:,-1]
X_train_all=np.concatenate((X_train_full,X_train_full,X_train_full,X_train_full,X_train_full,X_train_full,X_train_full,X_train_full,X_train_full,X_train_full), axis=0)
y_train_all=np.concatenate((y_train_full,y_train_full,y_train_full,y_train_full,y_train_full,y_train_full,y_train_full,y_train_full,y_train_full,y_train_full), axis=0)


# Nouvelle section

In [ ]:
X_train_all.shape

In [ ]:
X_test = test_data

In [ ]:
# Let's transform rows to images
X_test = test_data
X_train_all = X_train_all.reshape(-1, 28, 28,1)
X_test = X_test.reshape(-1, 28, 28,1)

X_valid, X_train = X_train_all[80000:] / 255., X_train_all[:80000] / 255.
y_valid, y_train = y_train_all[80000:], y_train_all[:80000]
X_test = X_test / 255.

In [ ]:
"""# **EarlyStopping**"""

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
                                                  restore_best_weights=True,
                                                 monitor="val_accuracy",
                                                 verbose=1)
reduce_lr_plateau = keras.callbacks.ReduceLROnPlateau(patience=2,monitor="val_accuracy",
                                                     verbose=1,
                                                     factor=0.05,
                                                      min_lr=0.000001
                                                     )
callbacks = [early_stopping_cb, reduce_lr_plateau]
                                                     

In [ ]:
batch_size = 1024
num_classes = 10
epochs = 100
learning_rate = 0.0001
model_name = 'model_HMAM'

model = Sequential()

model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(28, 28, 1)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=5, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(128, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=5, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(256, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Dense(128))
model.add(BatchNormalization())
model.add(Dense(10, activation='sigmoid'))

model.summary()

optimizer = RMSprop(lr=learning_rate)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
               optimizer= keras.optimizers.Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08),   
              #optimizer= keras.optimizers.Adamax(0.0001), ##keras.optimizers.SGD(0.001),
              metrics=["accuracy"])


In [ ]:
datagen_train = keras.preprocessing.image.ImageDataGenerator(
    #featurewise_center=True,
    #featurewise_std_normalization=True,
    rotation_range=5,
    width_shift_range=3,
    height_shift_range=3,
    horizontal_flip=False)




dataget_test = keras.preprocessing.image.ImageDataGenerator(
    #featurewise_center=True,
    #featurewise_std_normalization=True,
)

datagen_train.fit(X_train)
dataget_test.fit(X_train)

In [ ]:
history = model.fit(datagen_train.flow(X_train, y_train, batch_size=32),
          steps_per_epoch=len(X_train) / 32, epochs=200,
             validation_data=dataget_test.flow(X_valid, y_valid, batch_size=32),
                       callbacks=callbacks)



In [ ]:
# Now that we trained the model, we need to make a prediction on a test set, and save into csv
predictions = model.predict(X_test)
predictions_class = np.argmax(predictions, axis=1)
predictions_class



In [ ]:
df_submission = pd.DataFrame(predictions_class, columns = ["Category"])
df_submission["Id"] = df_submission.index
df_submission = df_submission[["Id", "Category"]]

In [ ]:
df_submission.head()

In [ ]:
df_submission.to_csv("baseline_HMAM38.csv", index=False)


# Nouvelle section

# Nouvelle section